In [1]:
#Install WandB
!pip install -q wandb

     |████████████████████████████████| 1.8 MB 5.2 MB/s 
     |████████████████████████████████| 144 kB 53.3 MB/s 
     |████████████████████████████████| 181 kB 61.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 


In [13]:
#Import all the Libraries
import sys
import pathlib
import numpy as np
import pandas as pd
import tensorflow as tf
import wandb
from PIL import Image
from tensorflow.keras.preprocessing import image_dataset_from_directory as image_generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, Model,Input
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
from tensorflow.keras.models import Model

In [3]:
#Path to the directory
Data_Directory = '/content/drive/MyDrive/inaturalist_12K/train'

In [4]:
#defining all the models
def INCEPTION_RESNET_V2(inputshape):
  return tf.keras.applications.InceptionResNetV2(input_shape=inputshape,weights='imagenet',include_top=False)
def INCEPTION_V3(inputshape):
  return tf.keras.applications.InceptionV3(input_shape=inputshape,weights='imagenet',include_top=False)
def RESNET50(inputshape):
  return tf.keras.applications.ResNet50(input_shape=inputshape,weights='imagenet',include_top=False)
def Xception(inputshape):
  return tf.keras.applications.Xception(input_shape=inputshape,weights='imagenet',include_top=False)
#Dictionary to get the model 
Model_List={"InceptionV3":INCEPTION_V3,"InceptionResNetV2":INCEPTION_RESNET_V2,"RESNET50":RESNET50,"Xception":Xception}

Reference:https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory

In [5]:
#Get training Data From The Directory
Training_Data = image_generator(
                      directory=Data_Directory,labels='inferred',  
                      label_mode='categorical',class_names=None,color_mode = 'rgb',
                      batch_size=32,image_size=(256, 256),shuffle=True,
                      seed=20,validation_split=0.1,
                      subset='training',follow_links=False,crop_to_aspect_ratio=False,)
# using only some percent of data
Training_Data = Training_Data.take(int(0.20* len(Training_Data)))


#preparing data augmentation
traindata_augmenter=tf.keras.preprocessing.image.ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    zca_epsilon=1e-06,
    rotation_range=25,
    width_shift_range=0.3,
    height_shift_range=0.3,
    brightness_range=None,
    shear_range=0.0,
    zoom_range=0.0,
    channel_shift_range=0.0,
    fill_mode='nearest',
    cval=0.0,
    horizontal_flip=True,
    vertical_flip=True,
    rescale=None,
    preprocessing_function=None,
    data_format='channels_last',
    validation_split=0.1,
    dtype=None)

Found 10017 files belonging to 10 classes.
Using 9016 files for training.


In [ ]:

Validation_Data = image_generator(
                      directory=Data_Directory,labels='inferred',
                      label_mode='categorical',class_names=None,color_mode ='rgb',
                      batch_size= 2,image_size=(256, 256),
                      shuffle=True,seed=20,validation_split = 0.1,
                      subset='validation', follow_links=False,crop_to_aspect_ratio=False,)

# using only some percent of data
Validation_Data = Validation_Data.take(int(0.20*len(Validation_Data)))

#preparing data augmentation
valdata_augmenter = ImageDataGenerator(validation_split = 0.1)

In [ ]:
#Generating Train Data and Test Data for augmented data
Train_Aug = traindata_augmenter.flow_from_directory(Data_Directory, shuffle = True, \
                                                         seed = 20, subset = 'training')
Val_Aug = valdata_augmenter.flow_from_directory(Data_Directory, shuffle = True,\
                                                     seed = 20, subset = 'validation')

Reference:https://www.tensorflow.org/guide/keras/custom_layers_and_models

In [8]:
#Subclass method for new layers and models
class Train_CNN(Model):
  def __init__(self,base_model_name,To_Tune=False,offset=10):
    super(Train_CNN, self).__init__()
    self.BaseModel_name=base_model_name
    self.BaseModel=Model_List[self.BaseModel_name]
    self.Shape_Image=(256,256,3)
    self.Train_Model=self.BaseModel(self.Shape_Image)

    if To_Tune == False:
      self.Train_Model.trainable=False

    else:
      self.Train_Model.trainable=True
      FineTune_till=len(self.Train_Model.layers)-offset
      for subLayer in self.Train_Model.layers[:FineTune_till]:
        subLayer.trainable=False
      

    self.Add_Avg_Pool=layers.GlobalAveragePooling2D()
    self.Add_Dense=layers.Dense(10)

  def call(self,inputs):
    temp=self.Train_Model(inputs)
    temp=self.Add_Avg_Pool(temp)
    return self.Add_Dense(temp)

In [ ]:
#Sample Run
#Uncomment to execute

"""
#Replace Base_Model with any of the given model
Parameters = sys.argv[1:]
Base_Model=Parameters[1]      #Available Models are {"InceptionV3","InceptionResNetV2","RESNET50","Xception"}
Tune=Parameters[3]
Offset=Parameters[5]
trained_model=Train_CNN(Base_Model,Tune,Offset)
trained_model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
trained_model.fit(Training_Data, epochs=1, validation_data=Validation_Data)
"""

In [ ]:
#Sample Run
#Uncomment to execute
#For Augmented Data
"""
#Replace Base_Model with any of the given model
Parameters = sys.argv[1:]
Base_Model=Parameters[1]      #Available Models are {"InceptionV3","InceptionResNetV2","RESNET50","Xception"}
Tune=Parameters[3]
Offset=Parameters[5]
trained_model=Train_CNN(Base_Model,Tune,Offset)
trained_model.compile(optimizer=tf.keras.optimizers.Adam(),
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])
trained_model.fit(Train_Aug, epochs=1, validation_data=Val_Aug)
"""

In [9]:
#Initializing the Sweep Functionality and parameters
Sconfig={'name':'MainPart2_Sweep','method':'grid'}
Sconfig['metric'] = {'name': 'val_acc', 'goal': 'maximize'}
Parameter_List={
    
    'Base_Model_Name':{'values':["InceptionV3","InceptionResNetV2","RESNET50","Xception"]},
    'Tune':{'values':[True,False]},
    'Data_Aug':{'values':[True,False]}
}
Sconfig['parameters'] = Parameter_List

In [10]:
def main(Base_Model_Name,Data_Aug,Tune):
    Base_Model=Train_CNN(Base_Model_Name,Tune)       
    Base_Model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics=['accuracy'])
    #if Data Augmentation is True
    if Data_Aug == True:
      Base_Model.fit(Train_Aug, steps_per_epoch=60, epochs=5,validation_data=Val_Aug, validation_steps=60,workers=10, callbacks = [wandb.keras.WandbCallback()])

    else:
      Base_Model.fit(Training_Data, epochs=5, validation_data=Validation_Data,callbacks = [wandb.keras.WandbCallback()])

In [11]:
#Driver Function
def CNN_PreTrain(config=Sconfig):
  with wandb.init(config=config):
    config = wandb.init().config
    BaseModelName=config.Base_Model_Name
    Tune=config.Tune
    Data_Aug=config.Data_Aug
    wandb.run.name = 'BM_{}_Aug_{}_Tune_{}'.format(BaseModelName,Data_Aug,Tune)
    main(BaseModelName,Data_Aug,Tune)

In [ ]:
#setting Up Sweep Agent
SweepId=wandb.sweep(Sconfig,project='DL-PartB')
wandb.agent(SweepId,function=CNN_PreTrain)